In [ ]:
!pip install pycaret
!pip install mlflow
!pip install explainerdashboard

In [ ]:
from google.colab import drive
import pandas as pd
import pycaret
from pycaret.classification import *

from sklearn.model_selection import train_test_split

drive.mount('/content/drive', force_remount=True)

import os

os.chdir('/content/drive/MyDrive/MY COMP333 PROJECT')

os.getcwd()

data = pd.read_csv('/content/drive/MyDrive/MY COMP333 PROJECT/final_df.csv')
data1 = data.copy()

unclean_data = pd.read_csv('/content/drive/MyDrive/MY COMP333 PROJECT/df_final_uncleaned.csv')
unclean_data1 = unclean_data.copy()


Mounted at /content/drive


In [ ]:
unclean_data1

,Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,All-Star
0,Al Horford,204,C,30,BOS,68,68,32.3,5.6,11.8,...,5.4,6.8,5.0,0.8,1.3,1.7,2.0,14.0,2016-2017,True
1,Allan Houston,181,SG,28,NYK,82,82,38.6,7.5,15.5,...,2.8,3.3,2.7,0.8,0.2,2.3,2.7,19.7,1999-2000,True
2,Amar'e Stoudemire,377,PF,27,PHO,82,82,34.6,8.6,15.4,...,6.1,8.9,1.0,0.6,1.0,2.6,3.4,23.1,2009-2010,True
3,Andre Drummond,115,C,23,DET,81,81,29.7,6.0,11.2,...,9.5,13.8,1.1,1.5,1.1,1.9,2.9,13.6,2016-2017,True
4,Andre Iguodala,217,SF,27,PHI,67,67,36.9,5.0,11.3,...,4.9,5.8,6.3,1.5,0.6,2.1,1.6,14.1,2010-2011,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,Ãmer AÅÄ±k,23,C,31,TOT,18,0,10.1,0.5,1.2,...,2.1,2.6,0.2,0.1,0.2,0.5,1.1,1.2,2017-2018,False
2372,Å arÅ«nas JasikeviÄius,211,SG-PG,30,TOT,63,3,15.4,2.0,5.1,...,0.9,1.1,2.7,0.4,0.0,1.4,1.2,6.1,2006-2007,False
2373,Å½an Tabak,394,C,30,IND,55,14,14.1,1.8,3.4,...,2.7,3.9,0.6,0.2,0.5,1.0,2.3,3.9,2000-2001,False
2374,Å½arko Äabarkapa,69,PF,24,GSW,61,0,8.3,1.1,2.9,...,1.2,1.8,0.3,0.2,0.1,0.6,1.4,3.3,2005-2006,False


In [ ]:
# only keeps first year out of the range
data['Year'] = data['Year'].str.split('-').str[0].astype(int)
unclean_data['Year'] = unclean_data['Year'].str.split('-').str[0].astype(int)

In [ ]:
# only keeps second year out of the range
data1['Year'] = data1['Year'].str.split('-').str[1].astype(int)
unclean_data1['Year'] = unclean_data1['Year'].str.split('-').str[1].astype(int)

We

In [ ]:
def pipline_running(data, try_num, **key_args):
  if not os.path.exists('/content/drive/MyDrive/MY COMP333 PROJECT/{}'.format(str(try_num))):
    os.mkdir('/content/drive/MyDrive/MY COMP333 PROJECT/{}'.format(str(try_num)))
    os.mkdir('/content/drive/MyDrive/MY COMP333 PROJECT/{}/lr'.format(str(try_num)))
    os.mkdir('/content/drive/MyDrive/MY COMP333 PROJECT/{}/ridge'.format(str(try_num)))
    os.mkdir('/content/drive/MyDrive/MY COMP333 PROJECT/{}/rf'.format(str(try_num)))
    os.mkdir('/content/drive/MyDrive/MY COMP333 PROJECT/{}/best_model'.format(str(try_num)))

  train, test = train_test_split(data, test_size=0.2, random_state=786)

  # identifying All-Star column as the target , excluded from training automatically
  clf1 = setup(train, target = 'All-Star', session_id=123, log_experiment=False, experiment_name='all-star', **key_args, verbose=True, fold=5)

  set_config('seed', 786)

  model_lr = create_model('lr', fold=10, verbose=False)
  model_ridge = create_model('ridge', fold=10, verbose=False)
  model_rf = create_model('rf', fold=10, verbose=False)

  def visualize_model(model, name):
    plot_model(model, 'confusion_matrix', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'class_report', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'learning', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'feature', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'pipeline', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'error', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'boundary', save='{}/{}'.format(str(try_num), name))
    plot_model(model, 'feature_all', save='{}/{}'.format(str(try_num), name))

  visualize_model(model_lr, 'lr')
  visualize_model(model_ridge, 'ridge')
  visualize_model(model_rf, 'rf')

  best = compare_models()

  # print(best)
  save_model(best, '/content/drive/MyDrive/MY COMP333 PROJECT/{}/{}'.format(str(try_num), 'best_model'))
  visualize_model(best, 'best_model')

  # print('Accuracy on unseen data: ')
  predictions = predict_model(best, data=test)
  predictions.to_csv('/content/drive/MyDrive/MY COMP333 PROJECT/{}/predictions.csv'.format(str(try_num)))

  return best, model_lr, model_ridge, model_rf


In [ ]:
# DATA - FIRST YEAR
# encoding (categorical) for position, team and year
# keeping outliers
# ignoring only the Players column, which is not a relevant feature for predictions
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':False, 'ignore_features': ['Player']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 0, **key_args)

,Description,Value
0,Session id,123
1,Target,All-Star
2,Target type,Binary
3,Original data shape,"(1188, 32)"
4,Transformed data shape,"(1188, 67)"
5,Transformed train set shape,"(831, 67)"
6,Transformed test set shape,"(357, 67)"
7,Ignore features,1
8,Numeric features,27
9,Categorical features,3


KeyboardInterrupt: 

In [ ]:
# DATA1 - SECOND YEAR
# encoding (categorical) for position, team and year
# keeping outliers
# ignoring only the Players column, which is not a relevant feature for predictions
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':False, 'ignore_features': ['Player']}
model, model_lr, model_ridge, model_rf = pipline_running(data1, 1, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9495,0.9708,0.7500,0.7241,0.7368,0.7089,0.7091


In [ ]:
# UNCLEAN_DATA
# encoding (categorical) for position, team and year
# ignoring only the Players column, which is not a relevant feature for predictions
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':False, 'ignore_features': ['Player']}
model, model_lr, model_ridge, model_rf = pipline_running(unclean_data, 1, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9832,0.9940,0.8966,0.8387,0.8667,0.8577,0.8583


In [ ]:
# clean data
# encoding only position, team and year
# removing outliers
# ignoring only the Players column, which is not a relevant feature for predictions
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':True, 'ignore_features': ['Player']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 2, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9562,0.9727,0.7857,0.7586,0.7719,0.7477,0.7479


In [ ]:
# unclean data
# encoding only position, team and year
# removing outliers
# ignoring only the Players column, which is not a relevant feature for predictions
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':True, 'ignore_features': ['Player']}
model, model_lr, model_ridge, model_rf = pipline_running(unclean_data, 3, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9706,0.9936,0.6897,0.8000,0.7407,0.7252,0.7275


In [ ]:
key_args = {'categorical_features': ['Tm', 'Pos'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 3, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.9562,0.9742,0.8214,0.7419,0.7797,0.7554,0.7566


In [ ]:
dashboard(model_rf, display_format ='inline')

Detected RandomForestClassifier model: Changing class type to RandomForestClassifierExplainer...
Note: model_output=='probability', so assuming that raw shap output of RandomForestClassifier is in probability space...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating shap interaction values... (this may take a while)
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating predictions...
Calculat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
key_args = {'categorical_features': ['Tm', 'Pos'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year', 'Rk']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 4, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.9562,0.9742,0.8214,0.7419,0.7797,0.7554,0.7566


In [ ]:
key_args = {'categorical_features': ['Tm', 'Pos'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year', 'Rk', 'FT']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 5, **key_args)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Linear Discriminant Analysis,0.9562,0.9742,0.8214,0.7419,0.7797,0.7554,0.7566


In [ ]:
# clean data
# only encoding team and position
# removing keeping outliers
key_args = {'categorical_features': ['Tm', 'Pos'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year', 'Rk', 'FT', 'G', 'PF']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 6, **key_args)

In [ ]:
key_args = {'categorical_features': ['Tm', 'Pos'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year', 'Rk', 'FT', 'G', 'PF']}
model, model_lr, model_ridge, model_rf = pipline_running(data, 6, **key_args)

,Description,Value
0,Session id,123
1,Target,All-Star
2,Target type,Binary
3,Original data shape,"(1188, 32)"
4,Transformed data shape,"(1188, 38)"
5,Transformed train set shape,"(831, 38)"
6,Transformed test set shape,"(357, 38)"
7,Ignore features,6
8,Numeric features,23
9,Categorical features,2


KeyboardInterrupt: 

In [ ]:
os.getcwd()

'/content/drive/MyDrive/MY COMP333 PROJECT'

In [ ]:
import numpy as np
import scipy.stats

import holoviews as hv
from holoviews.operation.timeseries import rolling, rolling_outlier_std

import colorcet

import bokeh.plotting
import bokeh.io

import holoviews as hv

import panel as pn
pn.extension()

hv.extension('bokeh')

bokeh.io.output_notebook()

from bokeh.plotting import figure, output_file, show

output_file('my_first_graph.html')


In [ ]:
data

,Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,All-Star
0,Al Horford,204,C,30,BOS,68,68,32.3,5.6,11.8,...,5.4,6.8,5.0,0.8,1.3,1.7,2.0,14.0,2016,True
1,Allan Houston,181,SG,28,NYK,82,82,38.6,7.5,15.5,...,2.8,3.3,2.7,0.8,0.2,2.3,2.7,19.7,1999,True
2,Allen Iverson,203,SG,32,DEN,82,82,41.8,8.7,19.0,...,2.4,3.0,7.1,2.0,0.1,3.0,1.3,26.4,2007,True
3,Alonzo Mourning,269,C,28,MIA,46,46,38.1,7.0,13.8,...,7.4,11.0,1.6,0.7,3.9,3.0,3.5,20.1,1998,True
4,Amar'e Stoudemire,377,PF,27,PHO,82,82,34.6,8.6,15.4,...,6.1,8.9,1.0,0.6,1.0,2.6,3.4,23.1,2009,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,ÃÂÃÂlex Abrines,1,SG,24,OKC,75,8,15.1,1.5,3.9,...,1.2,1.5,0.4,0.5,0.1,0.3,1.7,4.7,2017,False
1481,ÃÂÃÂscar Torres,400,SG,25,HOU,65,13,16.5,2.1,5.2,...,1.2,1.9,0.6,0.4,0.1,0.8,1.0,6.0,2001,False
1482,ÃÂ ÃÂ arÃÂ ÃÂ«nas JasikeviÃÂÃÂius,211,SG-PG,30,TOT,63,3,15.4,2.0,5.1,...,0.9,1.1,2.7,0.4,0.0,1.4,1.2,6.1,2006,False
1483,ÃÂ ÃÂ½an Tabak,381,C,27,TOT,57,34,17.3,2.5,5.3,...,2.2,3.7,0.8,0.4,0.7,1.1,2.9,5.4,1997,False


In [ ]:
# prompt: check the version of bokeh

print(bokeh.__version__)


3.3.4


In [ ]:
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap

x = list(range(1, 11))
y = list(range(1, 11))

source = ColumnDataSource(dict(x=x,y=y))

p = figure(width=300, height=300, title="Linear color map based on Y")

# use the field name of the column source
cmap = linear_cmap(field_name='y', palette="Spectral6", low=min(y), high=max(y))

p.scatter(x='x', y='y', color=cmap, size=15, source=source)

# pass the mapper's transform to the colorbar
color_bar = ColorBar(color_mapper=cmap['transform'], width=10)

p.add_layout(color_bar, 'right')

show(p)

In [ ]:
from bokeh.plotting import figure, output_file, show,ColumnDataSource
from bokeh.models import  ColumnDataSource,Range1d, LabelSet, Label


p = figure(title = "'Offensive vs. Defensive Eff'")
p.circle('GS','G', source=data, fill_alpha=0.6,size=10, )
p.xaxis.axis_label = 'GS'
p.yaxis.axis_label = 'G'
labels = LabelSet(x='GS', y='G', text='All-Star',text_font_size='5pt',
              x_offset=5, y_offset=5, source=ColumnDataSource(data))
p.add_layout(labels)
show(p)

In [ ]:
show(p)


In [ ]:
idf = data.interactive()

AttributeError: 'DataFrame' object has no attribute 'interactive'

In [ ]:
opts = dict(show_grid=True, frame_height=200, frame_width=350, color="#1f77b3")


def plot_normal_pdf(mu=0, sigma=1):
    x = np.linspace(-10, 10, 200)
    y = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

    return hv.Curve(data=(x, y), kdims=["x"], vdims=["f(x ; μ, σ)"]).opts(
        **opts
    )


plot_normal_pdf(0, 1)

ValueError: kdims argument expects a Dimension or list of dimensions, specified as tuples, strings, dictionaries or Dimension instances, not a Series type. Ensure you passed the data as the first argument.

In [ ]:
mu_slider = pn.widgets.FloatSlider(
    name="µ", start=-5, end=5, step=0.1, value=0
)
sigma_slider = pn.widgets.FloatSlider(
    name="σ", start=0.1, end=5, step=0.1, value=1
)


@pn.depends(mu_slider.param.value, sigma_slider.param.value)
def plot_normal_pdf(mu=0, sigma=1):
    x = np.linspace(-10, 10, 200)
    y = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

    return hv.Curve(data=(x, y), kdims=["x"], vdims=["f(x ; μ, σ)"]).opts(
        **opts
    )


widgets = pn.Column(
    pn.Spacer(height=30),
    mu_slider,
    pn.Spacer(height=15),
    sigma_slider,
    width=200,
)

pn.Row(plot_normal_pdf, pn.Spacer(width=15), widgets)

Row
    [0] ParamFunction(function, _pane=HoloViews, defer_load=False)
    [1] Spacer(width=15)
    [2] Column(width=200)
        [0] Spacer(height=30)
        [1] FloatSlider(end=5, name='µ', start=-5)
        [2] Spacer(height=15)
        [3] FloatSlider(end=5, name='σ', start=0.1, value=1)

In [ ]:
import panel as pn

symbol = pn.widgets.RadioButtonGroup(options=variables)
variable = pn.widgets.Select(options=variables)
rolling_window = pn.widgets.IntSlider(name='Rolling Window', value=10, start=1, end=365)

pn.extension()

pn.Column(symbol, variable, rolling_window)

Column
    [0] RadioButtonGroup(options=['Tm', 'FG%'], value='Tm')
    [1] Select(options=['Tm', 'FG%'], value='Tm')
    [2] IntSlider(end=365, name='Rolling Window', start=1, value=10)

In [ ]:
key_args = {'categorical_features': ['Pos', 'Tm', 'Year'], 'remove_outliers':False, 'ignore_features': ['Player', 'Year']}
model_lr, _, _ = pipline_running(data, 0, **key_args)

In [ ]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9675,0.9875,0.8042,0.8773,0.8299,0.8122,0.8185,1.3680
et,Extra Trees Classifier,0.9615,0.0000,0.6819,0.9154,0.7630,0.7435,0.7628,0.6280
rf,Random Forest Classifier,0.9567,0.0000,0.6694,0.8884,0.7458,0.7233,0.7415,0.6470
lightgbm,Light Gradient Boosting Machine,0.9567,0.0000,0.6792,0.8625,0.7358,0.7138,0.7321,1.0760
lda,Linear Discriminant Analysis,0.9555,0.9808,0.7667,0.8007,0.7740,0.7496,0.7554,0.3160
ridge,Ridge Classifier,0.9543,0.9812,0.6333,0.8974,0.7216,0.6985,0.7217,0.7270
ada,Ada Boost Classifier,0.9531,0.9548,0.6806,0.8162,0.7330,0.7079,0.7166,0.5580
gbc,Gradient Boosting Classifier,0.9531,0.9807,0.7194,0.7981,0.7478,0.7223,0.7285,0.8860
xgboost,Extreme Gradient Boosting,0.9458,0.0000,0.6222,0.7906,0.6858,0.6572,0.6684,0.3790
knn,K Neighbors Classifier,0.9339,0.0000,0.4236,0.8467,0.5548,0.5243,0.5652,0.3680


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model_lr = create_model('lr', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9762,0.9867,0.8889,0.8889,0.8889,0.8756,0.8756
1,0.9880,0.9967,0.8750,1.0000,0.9333,0.9267,0.9292
2,0.9639,0.9917,0.6250,1.0000,0.7692,0.7508,0.7752
3,0.9759,0.9850,0.7500,1.0000,0.8571,0.8443,0.8547
4,0.9518,0.9750,0.6250,0.8333,0.7143,0.6886,0.6971
5,0.9639,0.9950,0.7500,0.8571,0.8000,0.7802,0.7823
6,0.9759,0.9950,1.0000,0.8000,0.8889,0.8756,0.8824
7,0.9639,0.9767,0.7500,0.8571,0.8000,0.7802,0.7823
8,0.9277,0.9775,0.7778,0.6364,0.7000,0.6594,0.6637


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
predictions = predict_model(model_lr, data=test)
predictions.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9562,0.9727,0.7857,0.7586,0.7719,0.7477,0.7479


,Player,Rk,Pos,Age,Tm,G,GS,MP,FG,FGA,...,AST,STL,BLK,TOV,PF,PTS,Year,All-Star,prediction_label,prediction_score
1347,Terrence Ross,483,SG,30,ORL,63,0,23.000000,3.5,8.8,...,1.8,0.4,0.2,1.2,1.1,10.0,2021,False,0,0.9999
1232,Ron Harper,162,PG,37,LAL,47,46,24.200001,2.7,5.8,...,2.4,0.8,0.5,1.3,1.5,6.5,2000,False,0,0.9944
1183,Randy Brown,50,PG,32,BOS,54,35,22.900000,1.9,4.4,...,2.9,1.1,0.2,1.0,2.4,4.1,2000,False,0,0.9989
208,Anthony Goldwire,145,PG,26,DEN,82,32,27.000000,3.3,7.8,...,3.4,1.0,0.1,1.0,1.8,9.2,1997,False,0,0.9971
1266,Samaki Walker,392,C,26,LAL,67,39,18.600000,1.7,4.1,...,1.0,0.3,0.8,0.8,2.1,4.4,2002,False,0,0.9983


In [ ]:
model_ridge = create_model('ridge', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9615,0.9968,0.6000,1.0000,0.7500,0.7306,0.7586
1,0.9423,0.9670,0.6000,0.7500,0.6667,0.6355,0.6403
2,0.9423,0.9979,0.4000,1.0000,0.5714,0.5465,0.6132
3,0.9135,0.9745,0.1000,1.0000,0.1818,0.1673,0.3021
4,0.9808,0.9936,0.8000,1.0000,0.8889,0.8785,0.8851
5,0.9327,0.9851,0.3000,1.0000,0.4615,0.4365,0.5284
6,0.9231,0.9746,0.2727,1.0000,0.4286,0.4014,0.5011
7,0.9519,0.9902,0.5455,1.0000,0.7059,0.6822,0.7195
8,0.9423,0.9961,0.4545,1.0000,0.6250,0.5985,0.6535


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model_rf = create_model('rf', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9135,0.0000,0.1000,1.0000,0.1818,0.1673,0.3021
1,0.9231,0.0000,0.3000,0.7500,0.4286,0.3953,0.4436
2,0.9038,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9231,0.0000,0.3000,0.7500,0.4286,0.3953,0.4436
4,0.9615,0.0000,0.6000,1.0000,0.7500,0.7306,0.7586
5,0.9327,0.0000,0.3000,1.0000,0.4615,0.4365,0.5284
6,0.8942,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9423,0.0000,0.4545,1.0000,0.6250,0.5985,0.6535
8,0.8942,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]